<a href="https://colab.research.google.com/github/darwinagudeloh/FDL_2024-2_UDEA/blob/main/01_Preparacion_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from io import StringIO

In [4]:
# Lista de archivos en el repositorio
base_url = "https://raw.githubusercontent.com/darwinagudeloh/FDL_2024-2_UDEA/refs/heads/main/"
files = ["punto1.csv", "punto2.csv", "punto3.csv", "punto4.csv", "punto5.csv", "punto6.csv"]

In [5]:
# Crear un diccionario para almacenar los DataFrames de cada ubicación
data_dict = {}

for file in files:
    # Construir la URL de cada archivo
    file_url = base_url + file

    # Descargar el archivo desde GitHub
    response = requests.get(file_url)
    if response.status_code == 200:
        # Leer el contenido del archivo en un DataFrame
        df = pd.read_csv(StringIO(response.text), skiprows=9)  # skiprows=9 para omitir las primeras líneas del encabezado

        # Extraer el nombre del archivo sin extensión para usar como identificador
        location = file.split(".")[0]

        # Renombrar las columnas para que sean consistentes
        df = df.rename(columns={"YEAR": "year", "MO": "month", "DY": "day", "WS10M_MAX": f"WS10M_MAX_{location}"})

        # Combinar las columnas de año, mes y día en una sola columna de fecha
        df["Date"] = pd.to_datetime(df[["year", "month", "day"]])

        # Seleccionar solo las columnas necesarias
        df = df[["Date", f"WS10M_MAX_{location}"]]

        # Agregar el DataFrame al diccionario
        data_dict[location] = df
        print(f"{file} descargado y procesado con éxito.")
    else:
        print(f"Error al descargar {file}. Código de estado: {response.status_code}")

punto1.csv descargado y procesado con éxito.
punto2.csv descargado y procesado con éxito.
punto3.csv descargado y procesado con éxito.
punto4.csv descargado y procesado con éxito.
punto5.csv descargado y procesado con éxito.
punto6.csv descargado y procesado con éxito.


In [6]:
# Combinar los DataFrames de todas las ubicaciones en uno solo
combined_data = data_dict[list(data_dict.keys())[0]]  # Inicializar con el primer DataFrame
for location, df in data_dict.items():
    if location != list(data_dict.keys())[0]:  # Saltar el primer DataFrame ya incluido
        combined_data = pd.merge(combined_data, df, on="Date", how="outer")

# Mostrar las primeras filas del DataFrame combinado
print(combined_data.head())

        Date  WS10M_MAX_punto1  WS10M_MAX_punto2  WS10M_MAX_punto3  \
0 2014-01-01              1.93              1.35              1.01   
1 2014-01-02              2.03              1.55              1.45   
2 2014-01-03              1.29              1.49              1.30   
3 2014-01-04              2.19              1.28              1.30   
4 2014-01-05              1.20              1.02              1.29   

   WS10M_MAX_punto4  WS10M_MAX_punto5  WS10M_MAX_punto6  
0              2.17              2.55              1.41  
1              2.39              3.48              1.06  
2              1.78              2.66              1.50  
3              1.78              2.64              1.96  
4              2.39              3.58              1.46  


In [8]:
# Guarda el DataFrame combinado en Google Drive
#combined_data.to_csv('/content/drive/MyDrive/Fundamentos de Deep Learning/Proyecto/dataset/combined_wind_data.csv', index=False)